In [58]:
import boto3
import base64
import json
import fitz  # PyMuPDF
from pdf2image import convert_from_path
from PIL import Image
from pathlib import Path
import time
import numpy as np
from tqdm import tqdm

In [3]:
# =========================
# Configuración
# =========================

REGION = "us-east-1"
MODEL_ID = "amazon.nova-pro-v1:0"
MIN_CHARS_DIGITAL = 20   # umbral para detectar texto real
DPI = 400

## Preprocesado de la información

In [ ]:
# =========================
# Cliente Bedrock
# =========================

def aws_bedrock_client():
    return boto3.client("bedrock-runtime", region_name=REGION)

# =========================
# Utilidades imagen
# =========================

def load_image_for_nova(path: Path):
    with open(path, "rb") as f:
        raw = f.read()

    img = Image.open(path)
    fmt = img.format.lower()
    if fmt == "jpg":
        fmt = "jpeg"

    return {
        "bytes": base64.b64encode(raw).decode("utf-8"),
        "format": fmt
    }

# =========================
# OCR con Nova Pro
# =========================

def ocr_image_with_nova(client, image_path: Path, prompt: str):
    image = load_image_for_nova(image_path)
    body = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "text": prompt
                    },
                    {
                        "image": {
                            "format": image["format"],
                            "source": {
                                "bytes": image["bytes"]
                            }
                        }
                    }
                ]
            }
        ]
    }

    hora_inicio = time.time()

    response = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json"
    )

    hora_fin = time.time()

    result = json.loads(response["body"].read().decode("utf-8"))

    tokens_entrada = result.get("usage", {}).get("inputTokens", 0)
    tokens_salida = result.get("usage", {}).get("outputTokens", 0)

    result_dict = result.get("output", {}).get("message",{}).get("content", "")


    if isinstance(result_dict, list) and len(result_dict) == 1:
        texto_salida =  result_dict[0].get("text", "")
    elif isinstance(result_dict, list) and len(result_dict) > 1:
        texts = [
            item.get("text", "")
            for item in result_dict
            if "text" in item
        ]
        texto_salida =  "\n".join(texts)
    else:
        texto_salida =  None
    
    return {
        "output": texto_salida,
        "inputTokens": tokens_entrada,
        "outputTokens": tokens_salida,
        "elapsedTime": hora_fin - hora_inicio
    }

# =========================
# Detección de páginas
# =========================

def page_has_text(page, min_chars=MIN_CHARS_DIGITAL):
    text = page.get_text().strip()
    return len(text) >= min_chars

# =========================
# Procesamiento principal
# =========================

def process_pdf_mixed(pdf_path: str):
    pdf_path = Path(pdf_path)
    client = aws_bedrock_client()

    doc = fitz.open(pdf_path)

    digital_pages = []
    scanned_pages = []
    tokens_oct_input = []
    tokens_oct_output = []
    tiempo_empleado = []
    dimensiones_imagen = []

    # 1️⃣ Detectar tipo de página
    for i, page in enumerate(doc):
        # if page_has_text(page):
        #     digital_pages.append((i, page.get_text()))
        # else:
            scanned_pages.append(i)

    # 2️⃣ Texto digital (gratis)
    digital_results = [
        {
            "pagina": i + 1,
            "tipo": "digital",
            "texto": text
        }
        for i, text in digital_pages
    ]

    # 3️⃣ OCR solo páginas escaneadas
    ocr_results = []

    if scanned_pages:
        images = convert_from_path(
            pdf_path,
            dpi=DPI,
            first_page=min(scanned_pages) + 1,
            last_page=max(scanned_pages) + 1
        )

        page_map = {
            idx + min(scanned_pages): img
            for idx, img in enumerate(images)
        }

        for page_index in scanned_pages:
            img = page_map[page_index]
            img_path = Path(f"page_{page_index + 1}.png")
            img.save(img_path, "PNG")

            resultado_ocr = ocr_image_with_nova(
                client,
                img_path,
                prompt="Extrae todo el texto visible de la página con la mayor fidelidad posible."
            )

            ocr_results.append({
                "pagina": page_index + 1,
                "tipo": "escaneada",
                "texto": resultado_ocr["output"]
            })

            tokens_oct_input.append(resultado_ocr["inputTokens"])
            tokens_oct_output.append(resultado_ocr["outputTokens"])
            tiempo_empleado.append(resultado_ocr["elapsedTime"])
            dimensiones_imagen.append(img.size)

            img_path.unlink()  # limpiar archivo temporal

    # 4️⃣ Consolidar
    contexto = sorted(
        digital_results + ocr_results,
        key=lambda x: x["pagina"]
    )

    return {
        "contexto" : contexto,
        "tokensInputOCR": sum(tokens_oct_input),
        "tokensOutputOCR": sum(tokens_oct_output),
        "tiempoEmpleoOCR": sum(tiempo_empleado),
        "dimensionesImagenesOCR": dimensiones_imagen
    }

In [ ]:
ruta_proyecto = Path.cwd().parent

ruta_base = ruta_proyecto / "data/documents"

lista_archivos = [
    "EkosTextiles.pdf",
    "Ekosnegocios_TextilesSanPedro.pdf",
    "Extracto 11990032236 1990-06-07 297008.pdf",
    "documento_1765755270482.pdf"
]

lista_datos_contexto = []

for archivo in tqdm(lista_archivos):
    ruta_pdf = ruta_base / archivo

    contexto = process_pdf_mixed(ruta_pdf)

    lista_datos_contexto.append({
        "archivo": archivo,
        "datosContexto": contexto
    })

tokens_entrada_totales = np.sum([item['datosContexto']['tokensInputOCR'] for item in lista_datos_contexto])
tokens_salida_totales = np.sum([item['datosContexto']['tokensOutputOCR'] for item in lista_datos_contexto])
tiempo_total_ocr = np.sum([item['datosContexto']['tiempoEmpleoOCR'] for item in lista_datos_contexto])
altura_promedio = np.mean([
    dim[1]
    for item in lista_datos_contexto
    for dim in item['datosContexto']['dimensionesImagenesOCR']
])
ancho_promedio = np.mean([
    dim[0]
    for item in lista_datos_contexto
    for dim in item['datosContexto']['dimensionesImagenesOCR']
])

print(f"Tokens de entrada totales en OCR: {tokens_entrada_totales}")
print(f"Tokens de salida totales en OCR: {tokens_salida_totales}")
print(f"Tiempo total empleado en OCR (segundos): {tiempo_total_ocr:.2f}")
print(f"Dimensiones promedio de las imágenes OCR (ancho x alto): {ancho_promedio:.2f} x {altura_promedio:.2f} píxeles")

## Preguntas sobre la información

In [84]:
# =========================
# Preguntas sobre el documento
# =========================


def ask_about_document(contexto, pregunta):
    client = aws_bedrock_client()


    indicacion_base = "Tienes la siguiente información extraída de un documento PDF. Cada linea corresponde a una página:"

    body = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"text": indicacion_base},
                    {"text" : contexto},
                    {"text": pregunta}
                ],
            }
        ]
    }

    tiempo_inicio = time.time()

    response = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json",
    )

    tiempo_fin = time.time()

    result = json.loads(response["body"].read().decode("utf-8"))

    texto_respuesta = result.get("output", {}).get("message", {}).get("content", [])
    return {
        "respuesta": texto_respuesta,
        "tokensInput": result.get("usage", {}).get("inputTokens", 0),
        "tokensOutput": result.get("usage", {}).get("outputTokens", 0),
        "tiempoEmpleo": tiempo_fin - tiempo_inicio
    }

In [79]:
listado_contexto = []
for dato in lista_datos_contexto:
    contenido = dato['datosContexto']['contexto']

    lista_datos_archivo = [
        item["texto"] for item in contenido
    ]

    listado_contexto.extend(lista_datos_archivo)

listado_contexto_texto = "\n".join(listado_contexto)

print(listado_contexto_texto)

TEXTIL SAN PEDRO SA Valle De Los Chillos Vía Sangolquí Km. 3 Amaguaná El Cortijo Quito - Ecuador ( ) 233-5548 C1312.02 Fabricación de tejidos (telas) aterciopelados y de felpilla, tejidos de rizo, tejidos de gasa, etcétera. 2024 2023 Posición ventas 2024: 2790 Ingresos Totales: $7,631,376 Utilidad Bruta: $728,474 Impuesto Causado: $168,947 Utilidad/Ingresos: --- Guía de Negoci Empr
Empresa TEXTIL SAN PEDRO SA | Ekosnegocios https://ekosnegocios.com/empresa/textil-san-pedro-sa/ Cotizaciones por TradingView TEXTIL SAN PEDRO SA Valle De Los Chillos Vía Sangolquí Km. 3 Amaguana El Cortijo Quito - Ecuador ( ) 233-5548 2024 Posición ventas 2024: 2790 Ingresos Totales: $7,631,376 Utilidad Bruta: $728,474 Impuesto Causado: $168,947 Utilidad/Ingresos: --- 2023 CIU: C1312.02 Fabricación de tejidos (telas) aterciopelados y de felpilla, tejidos de rizo, tejidos de gasa, etcetera. Otras empresas del sector DELLTEX INDUSTRIAL SA TEXTILES TEXSA SA FABRILFAME S.A. FRANCELANA S.A. ASTRA C.A. Volver a G

In [85]:
respuesta = ask_about_document(
    listado_contexto_texto,
    "Resume el documento y menciona los puntos clave."
)

print("\n=== RESPUESTA FINAL ===")
print(respuesta)


=== RESPUESTA FINAL ===
{'respuesta': [{'text': '**Resumen del Documento:**\n\n**Empresa: TEXTIL SAN PEDRO S.A.**\n- **Ubicación:** Valle De Los Chillos, Vía Sangolquí Km. 3, Amaguaná, El Cortijo, Quito - Ecuador\n- **Contacto:** Teléfono ( ) 233-5548\n- **Actividad Principal:** Fabricación de tejidos (telas) aterciopelados y de felpilla, tejidos de rizo, tejidos de gasa, etc.\n\n**Datos Financieros 2024:**\n- **Posición de ventas:** 2790\n- **Ingresos Totales:** $7,631,376\n- **Utilidad Bruta:** $728,474\n- **Impuesto Causado:** $168,947\n- **Utilidad/Ingresos:** No especificado\n\n**Estado de Resultados Integral (En USD):**\n- **Ingresos de Actividades Ordinarias:** $7,332,127.32\n  - Venta de bienes: $7,014,528.75\n  - Prestación de servicios: $125,521.53\n  - Intereses: $341,851.93\n- **Ganancia Bruta:** $2,753,813.95\n- **Otros Ingresos:** $299,248.67\n- **Costo de Ventas y Producción:** $4,578,313.37\n- **Gastos:** $2,324,588.14\n  - Gastos de venta: $1,076,851.26\n  - Gastos ad

In [90]:
print(respuesta['respuesta'][0]['text'])

**Resumen del Documento:**

**Empresa: TEXTIL SAN PEDRO S.A.**
- **Ubicación:** Valle De Los Chillos, Vía Sangolquí Km. 3, Amaguaná, El Cortijo, Quito - Ecuador
- **Contacto:** Teléfono ( ) 233-5548
- **Actividad Principal:** Fabricación de tejidos (telas) aterciopelados y de felpilla, tejidos de rizo, tejidos de gasa, etc.

**Datos Financieros 2024:**
- **Posición de ventas:** 2790
- **Ingresos Totales:** $7,631,376
- **Utilidad Bruta:** $728,474
- **Impuesto Causado:** $168,947
- **Utilidad/Ingresos:** No especificado

**Estado de Resultados Integral (En USD):**
- **Ingresos de Actividades Ordinarias:** $7,332,127.32
  - Venta de bienes: $7,014,528.75
  - Prestación de servicios: $125,521.53
  - Intereses: $341,851.93
- **Ganancia Bruta:** $2,753,813.95
- **Otros Ingresos:** $299,248.67
- **Costo de Ventas y Producción:** $4,578,313.37
- **Gastos:** $2,324,588.14
  - Gastos de venta: $1,076,851.26
  - Gastos administrativos: $1,116,384.76
  - Gastos financieros: $131,352.12
- **Ganan